In [1]:
from root_numpy import root2array, tree2array
import ROOT

Welcome to JupyROOT 6.12/06


In [2]:
rfile = ROOT.TFile("TreeFile.root")
intree = rfile.Get('Tree')
intree.Print()
# and convert the TTree into an array
array = tree2array(intree,branches=["Gen","Reco","data"])
array.dtype.names = ('reco', "gen", "data")

print(array)

[(799.245  , 777.50684, 777.50684) (688.7374 , 685.90155, 685.90155)
 (386.379  , 365.84082, 365.84082) ... (368.45944, 369.37573, 369.37573)
 (409.72958, 417.1106 , 417.1106 ) (137.92084, 213.0208 , 213.0208 )]
******************************************************************************
*Tree    :Tree      : Tree                                                   *
*Entries :  1000000 : Total =        12035518 bytes  File  Size =   10778962 *
*        :          : Tree compression factor =   1.12                       *
******************************************************************************
*Br    0 :Reco      : Reco/F                                                 *
*Entries :  1000000 : Total  Size=    4011767 bytes  File Size  =    3596983 *
*Baskets :      126 : Basket Size=      32000 bytes  Compression=   1.11     *
*............................................................................*
*Br    1 :Gen       : Gen/F                                                  

In [3]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
df = pd.DataFrame(array)

#print(df)
content_reco ,bin,patches = plt.hist(df['reco'], bins=15)
plt.ylabel('# events')
plt.xlabel("reco")
plt.show()

content_gen ,bin,patches = plt.hist(df['gen'], bins=15)
print(content_gen)
plt.ylabel('# events')
plt.xlabel("gen")
plt.show()

plt.hist2d(df['reco'],df['gen'], bins=10, range=[[0, 1000], [0, 1000]],norm=LogNorm())
plt.ylabel('# events')
plt.colorbar()
plt.show()



[5.16000e+02 2.15860e+04 1.46504e+05 2.43194e+05 1.78346e+05 1.14436e+05
 8.02140e+04 6.10580e+04 4.81930e+04 3.91600e+04 3.29390e+04 2.36680e+04
 9.20300e+03 9.60000e+02 2.30000e+01]


In [4]:
# KERAS imports:

from __future__ import print_function
import keras


batch_size = 1000
num_classes = 20
NBins = num_classes
epochs = 10
#
# Define initial function:
#

/home/swieland/tools/root/lib/ROOT.py:318: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return _orig_ihook( name, *args, **kwds )
Using TensorFlow backend.


In [5]:
gen=df["gen"].values
print(gen)
print(gen.shape)
g=np.trunc(gen/200)
print(g)
reco=df["reco"].values
print(reco)
print(reco.shape)
r=np.trunc(reco/200)
print(r)
data=df["data"].values
data=np.trunc(data/200)
data = data.reshape(data.shape[0],1,1)
gcat = keras.utils.to_categorical(g,NBins)
from sklearn.model_selection import train_test_split
r = r.reshape(r.shape[0],1,1)
r_train, r_test, g_train, g_test = train_test_split(r, gcat, test_size=0.22, random_state=42)

[777.50684 685.90155 365.84082 ... 369.37573 417.1106  213.0208 ]
(1000000,)
[3. 3. 1. ... 1. 2. 1.]
[799.245   688.7374  386.379   ... 368.45944 409.72958 137.92084]
(1000000,)
[3. 3. 1. ... 1. 2. 0.]


In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras import backend as K

In [7]:
def prepareModel(nvar=1, NBins=10, kappa=8):
    ''' Prepare KERAS-based sequential neural network with for ML unfolding. 
        Nvar defines number of inputvariables. NBins is number of truth bins. 
        kappa is an empirically tuned parameter for the intermediate layer'''
    model = Sequential()
    model.add(Dense(nvar,activation='linear',input_shape=(nvar,1)))
    model.add(Flatten())
    model.add(Dense(kappa*NBins**2,activation='relu'))
    
    # model.add(Dropout(0.25))
    # model.add(Dense(2*NBins,activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(NBins,activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    return model
model = prepareModel(1,NBins)

In [8]:
h = model.fit(r_train,g_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(r_test,g_test))

Train on 780000 samples, validate on 220000 samples
Epoch 1/10
780000/780000 [==============================] - 23s 29us/step - loss: 0.6056 - acc: 0.8099 - val_loss: 0.5615 - val_acc: 0.8166
Epoch 2/10
780000/780000 [==============================] - 22s 28us/step - loss: 0.5508 - acc: 0.8171 - val_loss: 0.5509 - val_acc: 0.8166
Epoch 3/10
780000/780000 [==============================] - 22s 29us/step - loss: 0.5500 - acc: 0.8171 - val_loss: 0.5506 - val_acc: 0.8166
Epoch 4/10
780000/780000 [==============================] - 24s 30us/step - loss: 0.5500 - acc: 0.8171 - val_loss: 0.5496 - val_acc: 0.8166
Epoch 5/10
780000/780000 [==============================] - 22s 29us/step - loss: 0.5495 - acc: 0.8171 - val_loss: 0.5489 - val_acc: 0.8166
Epoch 6/10
780000/780000 [==============================] - 22s 28us/step - loss: 0.5492 - acc: 0.8171 - val_loss: 0.5527 - val_acc: 0.8166
Epoch 7/10
780000/780000 [==============================] - 25s 32us/step - loss: 0.5493 - acc: 0.8171 - val

2018-03-22 23:31:34.843612: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2


In [9]:
# save model,if needed
model.save("model.hdf5")

In [10]:
loss_and_metrics = model.evaluate(r_test, g_test, batch_size=128)
print(loss_and_metrics)

220000/220000 [==============================] - 3s 13us/step
[0.5505959611025724, 0.8166045454545454]


In [11]:
prob = model.predict(r_test, batch_size=128)
classes=prob.argmax(axis=-1)
np.set_printoptions(threshold=100000)
for i in range(20):    
    print(classes[i],g_test.argmax(axis=-1)[i])

0 0
2 2
3 3
1 1
1 0
0 0
2 2
0 0
1 0
0 1
0 0
0 0
0 0
2 2
1 1
0 0
2 2
0 0
1 1
0 0
